# GEDI_L2A Search and Visualize

Authors:

Date:

Description: This tutorial aims to provide information and code to help users get started working with the Global Ecosystem Dynamics Investigation (GEDI) Level 2A (GEDI02_A) product using the MAAP.

## Run This Notebook
To access and run this tutorial within MAAP's Algorithm Development Environment (ADE), please refer to the ["Getting started with the MAAP"](https://docs.maap-project.org/en/latest/getting_started/getting_started.html) section of our documentation.

Disclaimer: it is highly recommended to run a tutorial within MAAP's ADE, which already includes packages specific to MAAP, such as maap-py. Running the tutorial outside of the MAAP ADE may lead to errors.

## About the Data

## Additional Resources

## Importing and Installing Packages

 Information about the GEDI02_A product may be found at the [Data Set Landing Page](https://lpdaac.usgs.gov/products/gedi02_av002/). We will start by importing the packages which will allow us to search for, access, explore, and visualize GEDI02_A product data.

Note: This Jupyter notebook utilizes the `folium` and `pystac_client` packages. If you do not have these packages installed, uncomment the lines and run the following code block.

In [3]:
!pip install folium
!pip install pystac_client

  Using cached pystac_client-0.7.2-py3-none-any.whl (33 kB)
  Using cached pystac-1.8.2-py3-none-any.whl (175 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1


For this tutorial, we will import `boto3`, `folium`, `h5py`, `pandas`, `exists` from `os.path` and `Client` from `pystac_client` as shown in the following codeblock.

In [4]:
# Install packages
import boto3
import folium
import h5py
import os
import pandas as pd
from maap.maap import MAAP
from os.path import exists
from pystac_client import Client

## Search for GEDI_L2A Data

To search for data from the GEDI_L2A product using NASA's CMR, we invoke the `MAAP` constructor, setting the `maap_host` argument to `'api.maap-project.org'`.

In [10]:
# Invoke the MAAP using the MAAP host argument
maap = MAAP(maap_host='api.maap-project.org')

Unable to load config file from source maap.cfg 
Unable to load config file from source ./maap.cfg 
Unable to load config file from source /projects/maap.cfg 


Now we can use the `searchGranule` function to find granule data within the collection, using the collection short name "GEDI02_A". Note that we can use `searchGranule`’s `cmr_host` argument to specify a CMR instance external to MAAP and the `readable_granule_name` argument to find granules matching either granule UR or producer granule id (please see the [API documentation](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html) for more information). In order to download data from NASA's CMR, we will set a variable to the first result from the `results` we obtained.

In [11]:
# Search for granule data using CMR host name and collection short name, and readable_granule_name arguments
results = maap.searchGranule(
    cmr_host='cmr.earthdata.nasa.gov',
    short_name='GEDI02_A',
    readable_granule_name = "GEDI02_A_2021272190541_O15849_04_T03030_02_003_02_V002.h5")

We'll go ahead and create a new data directory (if it doesn't already exist), and download the first result into it.

In [ ]:
# set data directory
dataDir = './data'

# check if directory exists -> if directory doesn't exist, directory is created
if not os.path.exists(dataDir):
    os.mkdir(dataDir)
    
# Download first result
filename = results[0].getData()

If desired, the `print` function can be utilized to see the file name and directory.

In [12]:
# Print file directory
print(filename)

./GEDI02_A_2021272190541_O15849_04_T03030_02_003_02_V002.h5


## Explore

Now that we have downloaded the data, let's look into what it contains.

In [13]:
# Create variable containing info from the file we downloaded
gediL2A = h5py.File(filename, 'r')

GEDI_L2A data has data for 8 different beams. Let's create a list of beam names to help explore the data.

In [14]:
# Create list of beam names
beamNames = [g for g in gediL2A.keys() if g.startswith('BEAM')]
beamNames

['BEAM0000',
 'BEAM0001',
 'BEAM0010',
 'BEAM0011',
 'BEAM0101',
 'BEAM0110',
 'BEAM1000',
 'BEAM1011']

Now let's explore the information available for one of the beams (in this case 'BEAM0000').

In [15]:
# Get list of objects in the data pertaining to 'BEAM0000'
beam = beamNames[0]
gediL2A_objs = []
gediL2A.visit(gediL2A_objs.append)
gediSDS = [o for o in gediL2A_objs if isinstance(gediL2A[o], h5py.Dataset)]
[i for i in gediSDS if beam in i][0:20]

['BEAM0000/ancillary/l2a_alg_count',
 'BEAM0000/beam',
 'BEAM0000/channel',
 'BEAM0000/degrade_flag',
 'BEAM0000/delta_time',
 'BEAM0000/digital_elevation_model',
 'BEAM0000/digital_elevation_model_srtm',
 'BEAM0000/elev_highestreturn',
 'BEAM0000/elev_lowestmode',
 'BEAM0000/elevation_bias_flag',
 'BEAM0000/elevation_bin0_error',
 'BEAM0000/energy_total',
 'BEAM0000/geolocation/elev_highestreturn_a1',
 'BEAM0000/geolocation/elev_highestreturn_a2',
 'BEAM0000/geolocation/elev_highestreturn_a3',
 'BEAM0000/geolocation/elev_highestreturn_a4',
 'BEAM0000/geolocation/elev_highestreturn_a5',
 'BEAM0000/geolocation/elev_highestreturn_a6',
 'BEAM0000/geolocation/elev_lowestmode_a1',
 'BEAM0000/geolocation/elev_lowestmode_a2']

## Visualize

Now that we've seen the various labels within the /BEAM0000 group, let's use this information to visualize the GEDI orbit path for our scenes. To start, we shall get samples for various shots, the beam number, longitude, latitude, and quality flags. We can use these samples to create and display a `pandas` dataframe.

In [16]:
# Set variables for shot, beam number, longitude, latitude, and quality flag samples
lonSample, latSample, shotSample, qualitySample, beamSample = [], [], [], [], [] 
lats = gediL2A[f'{beamNames[0]}/lat_lowestmode'][()]
lons = gediL2A[f'{beamNames[0]}/lon_lowestmode'][()]
shots = gediL2A[f'{beamNames[0]}/shot_number'][()]
quality = gediL2A[f'{beamNames[0]}/quality_flag'][()]
for i in range(len(shots)):
    if i % 100 == 0:
        shotSample.append(str(shots[i]))
        lonSample.append(lons[i])
        latSample.append(lats[i])
        qualitySample.append(quality[i])
        beamSample.append(beamNames[0])
# Create a pandas dataframe containing the sample information
latslons = pd.DataFrame({'Beam': beamSample, 'Shot Number': shotSample, 'Longitude': lonSample, 
                         'Latitude': latSample, 'Quality Flag': qualitySample})
# Display the dataframe
latslons

,Beam,Shot Number,Longitude,Latitude,Quality Flag
0,BEAM0000,158490000400502383,52.337698,0.599011,0
1,BEAM0000,158490000400502483,52.367554,0.556129,0
2,BEAM0000,158490000400502583,52.396932,0.514800,0
3,BEAM0000,158490000400502683,52.426548,0.472746,0
4,BEAM0000,158490000400502783,52.456248,0.430695,0
...,...,...,...,...,...
1685,BEAM0000,158490000400670883,135.837810,-51.605016,0
1686,BEAM0000,158490000400670983,135.921291,-51.605653,0
1687,BEAM0000,158490000400671083,136.003206,-51.606299,0
1688,BEAM0000,158490000400671183,136.086629,-51.606741,0


We can now create a map of the orbit path using the dataframe that we have created and utilizing the `folium.Map` and `folium.Circle` functions. Include `map` in the code block to inspect the map which is created within the Jupyter notebook.

In [17]:
# Create a map
map = folium.Map(
    location=[
        latslons.Latitude.mean(),
        latslons.Longitude.mean()
    ], zoom_start=3, control_scale=True
)
# Add variables to the map
for index, location_info in latslons.iterrows():
    folium.Circle(
        [location_info["Latitude"], location_info["Longitude"]],
        popup=f"Shot Number: {location_info['Shot Number']}"
    ).add_to(map)
# Display map
map

References:

- https://lpdaac.usgs.gov/resources/e-learning/getting-started-gedi-l2a-version-2-data-python/
- https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html